# Learning to Predict the Next Robot Action

## 1. Data Analysis

In [1]:
#Load the data set

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

narratives = pd.read_csv('data/narrative2vec.csv', sep=';')

In [ ]:
narratives.info()

In [ ]:
narratives.head()

In [ ]:
import header_names

%matplotlib inline
import matplotlib.pyplot as plt

narratives[header_names.PARENT].value_counts().plot.pie(figsize=(10,10),autopct='%1.1f%%')


In [ ]:
#TODO Create a piechart for label 'Next'

<hr/>

# 2. Data Preparation

## 2.1 Filling Empty Cells

In [ ]:
#Shows all columns which have empty blanks
narratives.isna().any()

In [ ]:
def fill_empty_cells(data):
    filled_data = data.copy()
    
    filled_data[header_names.PARENT]= filled_data[header_names.PARENT].fillna('NoParent')
    #TODO Fill the rest of the remaining empty cells
    
    return filled_data

## 2.2 Transform Categorical Values to Numeric Values

### 2.2.1 One Hot Encoding

![alt text](img/OneHotEncoding.png "One Hot Encoding Example")

In [ ]:
def transform_categorial_to_one_hot_encoded(data):
    encoded_data = data.copy()
    
    encoded_parent_data = pd.get_dummies(encoded_data[header_names.PARENT], prefix='parent')
    encoded_data = pd.concat([encoded_data, encoded_parent_data],axis=1)
    
    #TODO Transform the rest of the categorial features into one hot encoded features
    
    return encoded_data
    

## 2.3 Data Cleaning

In [ ]:
def clean(data):
    cleaned_data = data.copy()
    
    #TODO Decide which columns are not required to be able to predict the next robot action
    
    cols = [header_names.PARENT]
    
    for col in cols:
        cleaned_data = cleaned_data.drop(col, 1)
    
    return cleaned_data

## 2.4 Data Preparation Pipeline

In [ ]:
def prepare_data(data):
    prepared_data = data.copy()
    
    prepared_data = fill_empty_cells(prepared_data)
    #TODO apply all preparation methods on prepare_data
    
    return prepared_data


## 2.5 Prepared Data Evaluation

In [ ]:
#TODO store the prepared narratives in a prepared_narratives variable and evalute them by printing them

In [ ]:
#TODO verifiy that the prepared narratives do not have any empty cells

<hr/>

# 3. Brief Introduction to Decision Trees

## 3.1 Classification Example 

In [20]:
import numpy as np

example_data_set = np.array([['object','context','goal'],
              ['bowl','setting-up','cupboard'],
              ['bowl','cleaning','dishwasher'],
              ['cereal','setting-up','cupboard'],
              ['cereal','cleaning','cupboard'],
              ['milk','setting-up','fridge'],
              ['milk','cleaning','fridge'],
              ['spoon','setting-up','drawer'],
              ['spoon','cleaning','dishwasher']])


example_data = pd.DataFrame(data=example_data_set[1:,0:],
                         columns=example_data_set[0,0:])

example_data

,object,context,goal
0,bowl,setting-up,cupboard
1,bowl,cleaning,dishwasher
2,cereal,setting-up,cupboard
3,cereal,cleaning,cupboard
4,milk,setting-up,fridge
5,milk,cleaning,fridge
6,spoon,setting-up,drawer
7,spoon,cleaning,dishwasher


In [21]:
encoded_object_data = pd.get_dummies(example_data['object'], prefix='object')
encoded_context_data = pd.get_dummies(example_data['context'], prefix='context')

example_data = pd.concat([example_data, encoded_object_data],axis=1)
example_data = pd.concat([example_data, encoded_context_data],axis=1)

example_data = example_data.drop('object',1)
example_data = example_data.drop('context',1)

example_data.sort_values('goal')

,goal,object_bowl,object_cereal,object_milk,object_spoon,context_cleaning,context_setting-up
0,cupboard,1,0,0,0,0,1
2,cupboard,0,1,0,0,0,1
3,cupboard,0,1,0,0,1,0
1,dishwasher,1,0,0,0,1,0
7,dishwasher,0,0,0,1,1,0
6,drawer,0,0,0,1,0,1
4,fridge,0,0,1,0,0,1
5,fridge,0,0,1,0,1,0


## 3.2 Preview of a Trained Decision Tree

![alt text](img/example_tree.png "Decision Tree for the Example")

## 3.3 CART (Classification and Regression Trees) Algorithm

### 3.4 Gini Impurity

$$G_i=1-\sum_{c=1}^n {p_{i,c}}^2$$

In [25]:
def get_gini(example_set):
    cupboard_instances = float(example_set['goal'].value_counts().get('cupboard',0))
    dishwasher_instances = float(example_set['goal'].value_counts().get('dishwasher',0))
    drawer_instances = float(example_set['goal'].value_counts().get('drawer',0))
    fridge_instances = float(example_set['goal'].value_counts().get('fridge',0))

    instances = np.array([cupboard_instances, 
                          dishwasher_instances,
                          drawer_instances,
                          fridge_instances])
    
    total_instances = instances.sum()

    return 1-np.power(np.divide(instances, total_instances),2).sum()


### 3.5 Cost function

$$J(k,t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$$

"We care about a large set with low impurity then a small set with high impurity"

In [23]:
def cost(feature,threshold, data):
    true_data = data[(data[feature] <= threshold)]
    false_data = data[ (data[feature] > threshold)]

    ginis = np.array([get_gini(true_data),get_gini(false_data)])
    weights = np.divide(np.array([float(true_data.shape[0]),float(false_data.shape[0])]),data.shape[0])

    return np.multiply(weights,ginis).sum()


### 3.6 Picking a Threshold

![alt text](img/midpoints.png "Midpoints Scale")

### 3.7 Determining the Root Node

In [26]:
cost_table_headers = ['J(k,t_k)', 'K', 't_k']
cost_table_rows = []

for col in list(example_data):
    if col != 'goal':
        cost_table_rows.append([cost(col,0.5, example_data),col,0.5])

cost_data = pd.DataFrame(data=cost_table_rows,
                         columns=cost_table_headers)

cost_data.sort_values('J(k,t_k)')

,"J(k,t_k)",K,t_k
2,0.458333,object_milk,0.5
1,0.541667,object_cereal,0.5
3,0.583333,object_spoon,0.5
4,0.625000,context_cleaning,0.5
5,0.625000,context_setting-up,0.5
0,0.666667,object_bowl,0.5


## 3.8 Advantages & Disadvantages of Decision Trees

### 3.8.1 Advantages

<ul>
  <li>White box models</li>
  <li>Can be used for classification and regression problems</li>
  <li>Even for multioutput tasks</li>
  <li>Can gives us insights about features</li> 
</ul> 

### 3.8.2 Disadvantages

<ul>
  <li>They are unstable</li>
  <li>Decision boundaries are parallel to the axis</li>
  <li>Overfit easily, if they are not limited by hyperparameters</li>
</ul> 

![alt text](img/rotations_set.png "Unstable Example")

![alt text](img/regularization.png "Overfitting")

<hr/>

# 4. Additional Machine Learning Theory

## 4.1 Cross-validation

![alt text](img/xval.png "Example for Cross Validation")

## 4.2 Confusion Matrix

![alt text](img/conf_matrix.png "Example for a binary confusion matrix")

## 4.3 Classification Measurements

$$ Accuracy = \frac{TP + TN}{TP + FP + FN +TN}$$

$$ Precision = \frac{TP}{TP+FP}$$

$$ Recall = \frac{TP}{TP+FN}$$

$$ F_1 = 2 * \frac{Precision*Recall}{Precision+Recall}$$

<hr/>

# 5. Train the Next Action Classifier

## 5.1 Splitting the Data Set

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(prepared_narratives, test_size=0.2, random_state=42)

In [ ]:
def split_data_in_features_and_labels(data):
    cols = list(data)
    
    features_cols = []
    labels_cols = []
    
    for col in cols:
        if not col.startswith('next'):
            features_cols.append(col)
        else:
            labels_cols.append(col)
            
    features = data[features_cols]
    labels = data[labels_cols]
    
    return (features, features_cols, labels, labels_cols)

In [ ]:
#Splitting the training set into features and labels
train_set_features, train_set_features_cols, train_set_labels, train_set_labels_cols \
    = split_data_in_features_and_labels(train_set)

train_set_labels.head()

In [ ]:
#TODO Split the test set into features and labels. Their variables should have the prefix test_set

## 5.2 Train and Tune the Model Simultaneously

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz


#TODO find parameters which achieve a F1 score over 90%
parameters = {'max_depth':range(1,9,1), 'max_leaf_nodes': range(2,21,1)}

classifier = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10, n_jobs=4)

classifier.fit(train_set_features, train_set_labels)

tree_model = classifier.best_estimator_

print (classifier.best_score_, classifier.best_params_)

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(tree_model, out_file='data/tree.dot', feature_names=train_set_features_cols, class_names=tree_model.classes_)
#dot -Tpng tree.dot -o tree.png

<hr/>

# 6. Evaluate the Next Action Classifier

In [ ]:
from sklearn.metrics import classification_report

y_true, y_pred = test_set_labels, tree_model.predict(test_set_features)
print classification_report(y_true, y_pred)


In [ ]:
from sklearn.metrics import confusion_matrix
from confusion_matrix_plotter import plot_confusion_matrix

conf_mat = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(20, 20))
plot_confusion_matrix(conf_mat, classes=tree_model.classes_,
                      title='Confusion matrix, for the next action classifier')

plt.show()

In [ ]:
#TODO Tell why the classifier cannot differ between AcquireGraspOfSomething and MovingToLocation during prediction
#Hint 1: Compare the features between thoses labels
#Hint 2: Analyze the trained decision tree

#Print all features of the label 'AcquireGraspOfSomething'
sub_narratives = narratives[(narratives.next == 'AcquireGraspOfSomething')]
sub_narratives[[header_names.PARENT, header_names.PREVIOUS, header_names.TYPE]]
